In [1]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from tools.data import DreemDatasets
from preprocessing import WignerVilleSpectrum, Compose
import matplotlib.pyplot as plt
from mtspec.util import signal_bursts
from matplotlib.cm import ScalarMappable
import torch
import torch.utils.data
from torchvision.models import vgg11
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
use_datasets = ['eeg_1']
seed = 1
batch_size = 4
use_cuda = False
lr = 0.1
momentum = 0.5
log_every = 10
n_epochs = 50

In [3]:
wigner_ville_spectrum = WignerVilleSpectrum(time_bandwidth=3.5)

dataset_transform_spectrum = {
    "eeg_1": wigner_ville_spectrum,
    #"eeg_2": wigner_ville_spectrum,
    #"eeg_3": wigner_ville_spectrum,
    #"eeg_4": wigner_ville_spectrum,
    #"eeg_5": wigner_ville_spectrum,
    #"eeg_6": wigner_ville_spectrum,
    # "eeg_7": wigner_ville_spectrum
}

## Charger les données

On est obligé de loader les données au fur et à mesure, sinon trop gros dataset (plus de 90Go !!)

In [ ]:
train_set, val_set = DreemDatasets('dataset/train.h5', 'dataset/train_y.csv', 
                                   split_train_val=0.8, seed=seed, keep_datasets=use_datasets, 
                                   transforms=dataset_transform_spectrum, load_lazy=True).get()

train_set.load_data("dataset/all/train_split")
val_set.load_data("dataset/all/val_split")

train_set.close()
val_set.close()

Loading data in memory...
5412 in 1 datasets to load
Loading dataset eeg_1 ...


In [ ]:
"""X, _, Y = train_set[6]
print(X.shape)
x = X.transpose(1, 2, 0)
x -= x.min()
x /= x.max()

plt.imshow(x.repeat())"""

In [ ]:
train_loader = torch.utils.data.DataLoader(train_set.torch_dataset(), 
                                           batch_size=batch_size, 
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(val_set.torch_dataset(), batch_size=batch_size)

In [ ]:
def get_model(n_classes):
    """
    Size of image at least 224x224
    """
    model = vgg11(pretrained=True)
    model_conv = nn.Sequential(*list(model.children())[:-1])
    for param in model_conv.parameters():
        param.requires_grad = False
    model.classifier = nn.Sequential(
        #nn.Linear(512 * 7 * 7, 4096),
        nn.Linear(141312, 4096),
        nn.ReLU(True),
        nn.Dropout(),
        nn.Linear(4096, 4096),
        nn.ReLU(True),
        nn.Dropout(),
        nn.Linear(4096, n_classes),
    )
    return model

model = get_model(n_classes=5)

if use_cuda:
    model.cuda()
    
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [ ]:
def transform_data(data50):
    data50 -= data50.min()
    data50 /= data50.max()
    data50 = data50.to(dtype=torch.float)
    data50 = data50.expand(-1, 3, -1, -1)
    data50 = nn.functional.interpolate(data50, (200, 1500))

    return data50

In [ ]:
def train_step(epoch):
    model.train()
    with tqdm(total=len(train_loader.dataset) / train_loader.batch_size) as t:
        for batch_id, (data_50hz, data_10hz, target) in enumerate(train_loader):
            if use_cuda:
                data_50hz, data_10hz, target = data_50hz.cuda(), data_10hz.cuda(), target.cuda()
            optimizer.zero_grad()
            
            data50 = transform_data(data_50hz)
            out50 = model(data50)
            
            criterion = nn.CrossEntropyLoss(reduction='elementwise_mean')
            loss = criterion(out50, target)

            loss.backward()
            optimizer.step()
            if batch_id % log_every == 0:
                t.set_description("Train - Epoch " + str(epoch))
                t.set_postfix_str("Loss: " + str(loss.data.item()))
            t.update()


In [ ]:
def val_step(epoch):
    model.eval()

    validation_loss = 0
    correct = 0
    with tqdm(total=len(val_loader.dataset) / val_loader.batch_size) as t:
        for batch_id, (data_50hz, data_10hz, target) in enumerate(val_loader):
            if use_cuda:
                data_50hz, data_10hz, target = data_50hz.cuda(), data_10hz.cuda(), target.cuda()
            
            data50 = transform_data(data_50hz)
            out50 = model(data50)
            
            criterion = nn.CrossEntropyLoss(reduction='elementwise_mean')
            loss = criterion(out50, target)
            
            validation_loss += loss.data.item()
            # get the index of the max log-probability
            pred = out50.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
            if batch_id % log_every == 0:
                t.set_description("Val - Epoch " + str(epoch))
                t.set_postfix_str("Loss: " + str(loss.data.item()))
            t.update()

        validation_loss /= len(val_loader.dataset) / batch_size
        print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            validation_loss, correct, len(val_loader.dataset),
            100. * correct / len(val_loader.dataset)))


In [ ]:
for epoch in range(n_epochs):
    train_step(epoch)
    val_step(epoch)